In [1]:
from time import time
import pandas as pd
from scipy.io.arff import loadarff 

import h2o
from h2o.automl import H2OAutoML

In [9]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from C:\Users\gocal\Desktop\G-Boun\cmpe\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\gocal\AppData\Local\Temp\tmpyqwtwqmk
  JVM stdout: C:\Users\gocal\AppData\Local\Temp\tmpyqwtwqmk\h2o_gocal_started_from_python.out
  JVM stderr: C:\Users\gocal\AppData\Local\Temp\tmpyqwtwqmk\h2o_gocal_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_gocal_og15mi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.497 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [27]:
data_path = "https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/product_backorders.csv"

# Load data into H2O
df = h2o.import_file(data_path)
print(df.describe())

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:19053
Cols:23

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
type,int,int,int,int,int,int,int,int,int,int,int,int,enum,int,real,real,int,enum,enum,enum,enum,enum,enum
mins,1111620.0,-1440.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,-99.0,-99.0,0.0,,,,,,
mean,2059552.760562641,376.36702881435997,7.706036161335188,48.27234556237863,182.9108276911772,344.7398309977431,497.79242114102766,56.11887891670601,168.53445651603428,333.53219965359784,504.25539285151945,48.84070750013119,,2.3114995013908577,-6.5198336220017845,-6.05393533826694,0.8917755734005144,,,,,,
maxs,3284775.0,730722.0,52.0,170920.0,479808.0,967776.0,1418208.0,186451.0,550609.0,1136154.0,1759152.0,85584.0,,13824.0,1.0,1.0,1440.0,,,,,,
sigma,663337.6456498681,7002.071628662684,6.7786650721241895,1465.9992102068286,4304.865591970627,8406.062155159249,12180.570042918363,1544.2177775482571,4581.340080221506,9294.566153218984,14184.14539565362,968.7738680675267,,110.24106014611976,25.975138766871872,25.18449715003253,23.033345417338797,,,,,,
zeros,0,1858,121,15432,12118,11136,10604,10278,8022,6864,6231,9909,,18601,474,401,18585,,,,,,
missing,0,0,1078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1113121.0,0.0,8.0,1.0,6.0,6.0,6.0,0.0,4.0,9.0,12.0,0.0,No,1.0,0.9,0.89,0.0,No,No,No,Yes,No,Yes
1,1113268.0,0.0,8.0,0.0,2.0,3.0,4.0,1.0,2.0,3.0,3.0,0.0,No,0.0,0.96,0.97,0.0,No,No,No,Yes,No,Yes
2,1113874.0,20.0,2.0,0.0,45.0,99.0,153.0,16.0,42.0,80.0,111.0,10.0,No,0.0,0.81,0.88,0.0,No,No,No,Yes,No,Yes


None


In [28]:
df.head()

sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
1.11312e+06,0,8,1,6,6,6,0,4,9,12,0,No,1,0.9,0.89,0,No,No,No,Yes,No,Yes
1.11327e+06,0,8,0,2,3,4,1,2,3,3,0,No,0,0.96,0.97,0,No,No,No,Yes,No,Yes
1.11387e+06,20,2,0,45,99,153,16,42,80,111,10,No,0,0.81,0.88,0,No,No,No,Yes,No,Yes
1.11422e+06,0,8,0,9,14,21,5,17,36,43,0,No,0,0.96,0.98,0,No,No,No,Yes,No,Yes
1.11482e+06,0,12,0,31,31,31,7,15,33,47,2,No,3,0.98,0.98,0,No,No,No,Yes,No,Yes
1.11545e+06,55,8,0,216,360,492,30,108,275,340,51,No,0,0,0,0,No,No,Yes,Yes,No,Yes
1.11562e+06,-34,8,0,120,240,240,83,122,144,165,33,No,0,1,0.97,34,No,No,No,Yes,No,Yes
1.11645e+06,4,9,0,43,67,115,5,22,40,58,4,No,0,0.69,0.68,0,No,No,No,Yes,No,Yes
1.11683e+06,2,8,0,4,6,9,1,5,6,9,2,No,0,1,0.95,0,No,No,No,Yes,No,Yes
1.11687e+06,-7,8,0,56,96,112,13,30,56,76,0,No,0,0.97,0.92,7,No,No,No,Yes,No,Yes


In [4]:
raw_data = loadarff('../test/data/InsectSound_TRAIN.arff')
df_data = pd.DataFrame(raw_data[0])

In [11]:
hf = h2o.H2OFrame(df_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
hf.describe()


Rows:25000
Cols:601

att1                    att2                  att3                    att4                    att5                    att6                    att7                     att8                    att9                    att10                   att11                   att12                   att13                   att14                   att15                   att16                   att17                   att18                   att19                   att20                   att21                  att22                    att23                   att24                    att25                   att26                   att27                   att28                   att29                   att30                   att31                    att32                   att33                   att34                    att35                  att36                    att37                    att38                    att39                   att40                  att41                  att42                   att43                   att44                   att45                    att46                    att47                  att48                    att49                    att50                    att51                    att52                    att53                    att54                    att55                   att56                    att57                  att58                    att59                   att60                   att61                   att62                  att63                   att64                   att65                   att66                   att67                    att68                   att69                    att70                   att71                   att72                   att73                   att74                   att75                    att76                   att77                   att78                   att79                    att80                    att81                   att82                    att83                    att84                   att85                  att86                    att87                   att88                   att89                  att90                 att91                  att92                   att93                   att94                   att95                   att96                   att97                   att98                   att99                   att100                  att101                  att102                  att103                  att104                  att105                  att106                  att107                  att108                  att109                  att110                  att111                  att112                  att113                  att114                  att115                  att116                  att117                  att118                 att119                   att120                  att121                  att122                  att123                  att124                  att125                  att126                  att127                 att128                  att129                  att130                   att131                  att132                  att133                  att134                  att135                  att136                  att137                  att138                  att139                 att140                att141                 att142                   att143                  att144                   att145                  att146                  att147                   att148                att149                  att150                 att151                  att152                  att153                 att154                 att155                 att156                 att157                 att158                 att159                  att160                 att161                  att162                 att163                 att164                 att165                 att166                 att167             

In [20]:
hf[hf.col_names[0]].astype(int).describe()

AttributeError: 'H2OFrame' object has no attribute 'astype'

In [62]:
h2o.init()
df_data = pd.read_csv("../secondary_data.csv", delimiter = ";")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 16 mins
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_goktug_30ldsj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.709 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,32
H2O_cluster_status:,"locked, healthy"


In [63]:
hf = h2o.H2OFrame(df_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [64]:
hf.describe()

Rows:61069
Cols:21

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
type,enum,real,enum,enum,enum,enum,enum,enum,enum,real,real,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum
mins,,0.38,,,,,,,,0.0,0.0,,,,,,,,,,
mean,,6.7338544924593355,,,,,,,,6.5815382600010235,12.149410011626216,,,,,,,,,,
maxs,,62.34,,,,,,,,33.92,103.91,,,,,,,,,,
sigma,,5.264844947062989,,,,,,,,3.370016861289655,10.035955043117529,,,,,,,,,,
zeros,,0,,,,,,,,1059,1059,,,,,,,,,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,p,15.26,x,g,o,f,e,nan,w,16.95,17.09,s,y,w,u,w,t,g,nan,d,w
1,p,16.6,x,g,o,f,e,nan,w,17.99,18.19,s,y,w,u,w,t,g,nan,d,u
2,p,14.07,x,g,o,f,e,nan,w,17.8,17.74,s,y,w,u,w,t,g,nan,d,w


In [65]:
y = "class"
x = hf.columns

In [66]:
s = time()
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x = x, y = y, training_frame = hf)

print("Execution Time :", time() - s)

AutoML progress: |███████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_a57d067bff34b49561aaf3600dc54f52> was cancelled by the user.

In [37]:
lb = aml.leaderboard
lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_7_20221206_221349,1.10923,1.23038,0.751965,0.258596,1.23038
StackedEnsemble_BestOfFamily_1_AutoML_7_20221206_221349,1.13126,1.27975,0.771815,0.265095,1.27975
GBM_4_AutoML_7_20221206_221349,1.1721,1.37381,0.805219,0.275647,1.37381
XGBoost_3_AutoML_7_20221206_221349,1.17965,1.39157,0.824785,0.273234,1.39157
GBM_1_AutoML_7_20221206_221349,1.18082,1.39433,0.828727,0.278884,1.39433
GBM_3_AutoML_7_20221206_221349,1.18552,1.40546,0.818683,0.278678,1.40546
GBM_2_AutoML_7_20221206_221349,1.19477,1.42748,0.828012,0.281299,1.42748
DRF_1_AutoML_7_20221206_221349,1.20444,1.45066,0.826033,0.284196,1.45066
XRT_1_AutoML_7_20221206_221349,1.20611,1.45469,0.82767,0.284439,1.45469
XGBoost_2_AutoML_7_20221206_221349,1.21013,1.46441,0.827073,0.279038,1.46441
